## Entity Relationship Diagram of the database to populate

<img src="EntityRelationshipDiagram_dbmgmtProject2.png" alt="drawing" width="1000"/>

## Creating the city table

In [292]:
# create city dataframe
import uuid
import pandas as pd

data = {"city_id":[],"city_name":[]}

cityNames = ["Provo", "Spanish Fork", "Payson", "Springville", "Orem", "Salt Lake City",
             "Draper", "Lehi", "American Fork", "Saratoga Springs", "Lindon", "Pleasant Grove",
              "Vineyard", "Heber", "Midway", "Park City", "Highland", "Alpine", "Sandy", "Bluffdale"
              , "West Jordan", "Santaquin"]

# using 8 character id for id because appsheet will do this 
for city in cityNames:
    data["city_id"].append(str(uuid.uuid4())[-8:])
    data["city_name"].append(city)


dfCities = pd.DataFrame(data)

# add alternative names if any
alternativeNames = {"Salt Lake City" : "SLC", "American Fork" : "AF"}
for k, v in alternativeNames.items():
    dfCities.loc[len(dfCities) + 1] = [dfCities[dfCities.city_name == k]["city_id"].item(), v ]

# add all alternative names to all city names list for use later in cleaning
allCityNames = cityNames
for item in list(alternativeNames.values()):
    allCityNames.append(item)

dfCities

,city_id,city_name
0,e9c03db7,Provo
1,39cddece,Spanish Fork
2,e6ec6a8f,Payson
3,074cbda1,Springville
4,86e7b951,Orem
5,7bf336c7,Salt Lake City
6,bbac1723,Draper
7,bd85c73f,Lehi
8,e4848169,American Fork
9,5c1f3ef8,Saratoga Springs


## Scraping Utah.com events

In [293]:
#Create events dataframe
import requests 
from bs4 import BeautifulSoup
import time
import pandas as pd
import uuid

url = "https://utah.com/events-in-utah?month=december&sort=upcoming&where=provo&year=2021"
res = requests.get(url)
time.sleep(.02)
soup = BeautifulSoup(res.text, 'html.parser')
data = {"event_id":[],"name":[],"description":[],"city_name":[],"dates":[],"url":[]}
for item in soup.find(name="div", attrs={"class":"promotions__grid"}):

    item2 = item.findChild(attrs={"class":"promotions__detail"})
    name = item2.findChild(attrs={"class":"promotions__heading heading heading--primary"}).text.strip()
    description = item2.findChild(attrs={"class":"promotions__description"}).text.strip()
    city_name = item2.findChild(attrs={"class":"promotions__location"}).text.strip()
    dates = item2.findChild(attrs={"class":"promotions__expires heading heading--primary heading--emphasis"}).text.strip()
    url = "https://utah.com" + item["href"]

    data["event_id"].append(str(uuid.uuid4())[-8:])
    data["name"].append(name)
    data["description"].append(description)
    data["city_name"].append(city_name)
    data["dates"].append(dates)
    data["url"].append(url)
    
dfEvents = pd.DataFrame(data)
dfEvents


,event_id,name,description,city_name,dates,url
0,914df89e,Cedar City Downtown Farmer's Market,Downtown Year Round Farmer’s Saturday Market held 9:00 am – 1:00 p...,Cedar City,Year round,https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...
1,9bf26cf6,Tuacahn Saturday Market,Enjoy the beautiful red rock surroundings at the Tuacahn market fe...,St. George,"Every Saturday, year-round",https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...
2,34a3d4cf,A Christmas Story - The Musical,This hilarious musical is sure to warm your heart and tickle your ...,St. George,"NOV 26 - DEC 22, 2021",https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...
3,c0b9b444,Kayak Workshop on the Green River,*Kayak Desolation Canyon!\n*Learn a new skill & enjoy an active va...,Desolation Canyon,"August 1-6, 2022",https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...
4,d78f7a1d,Saturday Morning Nature Walk,"Learn about the beautiful wetlands, native plant and animal specie...",Park City,"Saturdays, through December 18",https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...
5,dd0f8a10,Luminaria: Experience the Light,This exciting mile walk is filled with thousands of programmed lum...,Lehi,"November 18 - Jan 1, 2022",https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...
6,3ee09b1b,Spanish Fork Festival of Lights,Located at Canyon View Park. Admission is $8 per single-family veh...,Spanish Fork,"Nov 25, 2021 - Jan 1, 2022",https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...
7,8aed58a4,Elf Displays & Elf Scavenger Hunt,Gardner Village brings you another year of fun from the North Pole...,Gardner Village,"Nov 20 - Dec 30, 2021",https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...
8,3f7f511d,Heber Valley Polar Express Train,Join us on this fun and traditional train ride to the North Pole! ...,Heber,Nov 29 - Dec 23,https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...
9,65aae7a2,Holiday Window Stroll at The Grand America,Experience the holiday windows! Join Clara and the Nutcracker for ...,Salt Lake City,Now through January 1st,https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...


In [294]:
# Final cleaning of event table 
from datetime import datetime
# replace cities with city id
for city in allCityNames:
    dfEvents.loc[dfEvents['city_name'].str.contains(city), 'city_name'] = city
dfEvents_2 = dfEvents.merge(dfCities, on='city_name',)
dfEvents_2.drop(columns="city_name",inplace=True)

dfEvents_2.set_index("event_id",inplace=True)

def imputeYear(dTime):
    if datetime.now().month <= dTime.month <= 12:
        dTime = dTime.replace(year = datetime.now().year)
    else:
        dTime = dTime.replace(year = datetime.now().year + 1)
    return dTime

def convertToDatetime(dfColumn, error = "pass"):
    for index, date in dfColumn.items():
        try:
            # try to convert to date time. Because there are many formats in the column, some of these will fail
            dTime = datetime.strptime(date.strip(),"%b %d, %Y")
            dfColumn.at[index] = dTime
        except:
            try:
                dTime = datetime.strptime(date.strip(),"%B %d")
                dTime = imputeYear(dTime)
                dfColumn.at[index] = dTime
            except:
                try:
                    dTime = datetime.strptime(date.strip(),"%b %d")
                    dTime = imputeYear(dTime)
                    dfColumn.at[index] = dTime
                except:
                    try:
                        dTime = datetime.strptime(date.strip(),"%B")
                        dTime = imputeYear(dTime)
                        dfColumn.at[index] =dTime
                    except:
                        try:
                            dTime = datetime.strptime(date.strip(),"%b. %d, %Y")
                            dfColumn.at[index] =dTime
                        except:
                            try:
                                dTime = datetime.strptime(date.strip(),"%B %dst")
                                dTime = imputeYear(dTime)
                                dfColumn.at[index] =dTime
                            except:
                                try:
                                    dTime = datetime.strptime(date.strip(),"%B %dnd")
                                    dTime = imputeYear(dTime)
                                    dfColumn.at[index] =dTime
                                except:
                                    try:
                                        dTime = datetime.strptime(date.strip(),"%B %drd")
                                        dTime = imputeYear(dTime)
                                        dfColumn.at[index] =dTime
                                    except:
                                        try:
                                            dTime = datetime.strptime(date.strip(),"%B %dth")
                                            dTime = imputeYear(dTime)
                                            dfColumn.at[index] =dTime
                                        except:
                                            try:
                                                dTime = datetime.strptime(date.strip(),"%b. %d")
                                                dTime = imputeYear(dTime)
                                                dfColumn.at[index] =dTime
                                            except:
                                                try:
                                                    dTime = datetime.strptime(date.strip(),"%b. %dth")
                                                    dTime = imputeYear(dTime)
                                                    dfColumn.at[index] =dTime
                                                except:
                                                    try:
                                                        dTime = datetime.strptime(date.strip(),"%A, %B %d")
                                                        dTime = imputeYear(dTime)
                                                        dfColumn.at[index] =dTime
                                                    except:
                                                        if error == "pass":
                                                            pass
                                                        elif error == "none":
                                                            dfColumn.at[index] = ""
                                                        elif error == "now":
                                                            dfColumn.at[index] = datetime(year = datetime.now().year,month = datetime.now().month,day = datetime.now().day)
    return dfColumn

dfEvents_2['dates'] = dfEvents_2['dates'].str.replace("through","-")
dfEvents_2[['start_date', 'end_date']] = dfEvents_2['dates'].str.split('-', 1, expand=True)

# Clean dates
dfEvents_2.end_date = convertToDatetime(dfEvents_2.end_date, "none")
dfEvents_2.start_date = convertToDatetime(dfEvents_2.start_date, "now")
dfEvents_2.drop(columns="dates", inplace=True)

# values for creating foreign keys and data for source id table
sourceidUtah = str(uuid.uuid4())[-8:]
dfEvents_2["source_id"] = sourceidUtah
urlUtah = "https://utah.com/events-in-utah"
nameUtah = "utah.com"

dfEvents_2



,name,description,url,city_id,start_date,end_date,source_id
event_id,,,,,,,
d78f7a1d,Saturday Morning Nature Walk,"Learn about the beautiful wetlands, native plant and animal specie...",https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...,5f3b344b,2021-12-10 00:00:00,2021-12-18 00:00:00,bbdadd3b
56758989,Sundance Film Festival,"The 10-day festival brings more than 40,000 people to Park City ea...",https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...,5f3b344b,2022-01-20 00:00:00,,bbdadd3b
dd0f8a10,Luminaria: Experience the Light,This exciting mile walk is filled with thousands of programmed lum...,https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...,bd85c73f,2022-11-18 00:00:00,2022-01-01 00:00:00,bbdadd3b
3ee09b1b,Spanish Fork Festival of Lights,Located at Canyon View Park. Admission is $8 per single-family veh...,https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...,39cddece,2021-11-25 00:00:00,2022-01-01 00:00:00,bbdadd3b
3f7f511d,Heber Valley Polar Express Train,Join us on this fun and traditional train ride to the North Pole! ...,https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...,2dc1839d,2022-11-29 00:00:00,2021-12-23 00:00:00,bbdadd3b
65aae7a2,Holiday Window Stroll at The Grand America,Experience the holiday windows! Join Clara and the Nutcracker for ...,https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...,7bf336c7,2021-12-10 00:00:00,2022-01-01 00:00:00,bbdadd3b
5a2ce057,Christmas at the Ranch,"You'll see Santa's Workshop, Santa's Supply Train, the Candy Cane ...",https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...,7bf336c7,2022-11-26 00:00:00,2021-12-23 00:00:00,bbdadd3b
50cbb998,Beyond Van Gogh: The Immersive Experience,"While journeying through Beyond Van Gogh, guests witness over 300 ...",https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...,7bf336c7,2021-12-10 00:00:00,2021-12-31 00:00:00,bbdadd3b
a000289b,Dreamscapes - Immersive Art Exhibit,Don’t miss this limited opportunity to dream with your eyes open a...,https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...,7bf336c7,2021-12-10 00:00:00,,bbdadd3b


## Scrape BYU.edu events

In [295]:


res = requests.get("https://calendar.byu.edu/events?f0=4") # next 30 days of events
time.sleep(.2)
soup = BeautifulSoup(res.text)

parentOfEvents = soup.find(name="div", attrs={"class":"container full-width list list-3 p-0"})
data = {"event_id":[],"name":[],"description":[],"url":[],"city_id":[],"dates":[]}
for event in parentOfEvents.findAll(attrs={"class":"SearchSnippet SearchSnippet-GridItem"}):
    try:
        name = event.find(attrs={"class":"SearchSnippet-GridItem-title promo-title"}).text.strip()
    except:
        name = ""
        pass
    try:
        url = event.find(attrs={"class":"SearchSnippet-GridItem-title promo-title"}).contents[1]["href"].strip()
    except:
        url = ""
        pass
    try:
        dates = event.find(attrs={"class":"SearchSnippet-GridItem-eventDate"}).contents[1].text.strip()
    except:
        dates = ""
        pass
    try:
        description = event.find(attrs={"class":"SearchSnippet-GridItem-description promo-description"}).text.strip()
    except:
        description = ""
        pass
    data["event_id"].append(str(uuid.uuid4())[-8:])
    data["dates"].append(dates)
    data["name"].append(name)
    data["description"].append(description)
    data["url"].append(url)
data["city_id"] = [""]*len(data["url"])
dfEvents2 = pd.DataFrame(data)

#all events are in provo, so add the provo city id
dfEvents2["city_id"] = dfCities[dfCities.city_name == "Provo"]["city_id"].values[0]

# split dates into two columns
dfEvents2[['start_date', 'end_date']] = dfEvents2['dates'].str.split('-', 1, expand=True)
# Clean dates
dfEvents2.end_date = convertToDatetime(dfEvents2.end_date, "none")
dfEvents2.start_date = convertToDatetime(dfEvents2.start_date, "now")
dfEvents2.drop(columns="dates", inplace=True)

# values for creating foreign keys and data for source id table
sourceidBYU = str(uuid.uuid4())[-8:]
urlBYU = "https://calendar.byu.edu/events"
nameBYU = "byu"

#all events had the same source: byu. 
dfEvents2["source_id"] = sourceidBYU 

dfEvents2.set_index("event_id",inplace=True)
dfEvents2

,name,description,url,city_id,start_date,end_date,source_id
event_id,,,,,,,
f0058648,ACT Prep Online Weekend May 28-31 (Memorial Day weekend),BYU ACT Prep is jam-packed with proven test-taking strategies that...,https://calendar.byu.edu/conferences/act-prep-online-weekend-may-2...,e9c03db7,2022-05-28 00:00:00,2022-05-31 00:00:00,05729b31
c0e4a0bb,ACT Prep Online Weekend September 3-6 (Labor Day weekend),BYU ACT Prep is jam-packed with proven test-taking strategies that...,https://calendar.byu.edu/conferences/act-prep-online-weekend-septe...,e9c03db7,2022-09-03 00:00:00,2022-09-06 00:00:00,05729b31
58217c28,Bagel Bash,Don't miss our semi-annual event to kick off your finals prep with...,https://calendar.byu.edu/Education/bagel-bash-2021-12-10,e9c03db7,2021-12-10 00:00:00,,05729b31
922508f4,Learn About Model United Nations,Come learn about BYU’s award-winning Model United Nations program ...,https://calendar.byu.edu/Education/learn-about-model-united-nation...,e9c03db7,2021-12-14 00:00:00,,05729b31
b5ad38b4,Passing the Foreign Service Exam,Interested in a career in the Foreign Service? Have questions abou...,https://calendar.byu.edu/Education/passing-the-foreign-service-exa...,e9c03db7,2021-12-16 00:00:00,,05729b31
29a3ec5b,W Basketball vs. Washington State,"Marriott Center, Provo, UT",https://calendar.byu.edu/athletics/w-basketball-vs-washington-stat...,e9c03db7,2021-12-18 00:00:00,,05729b31
77c3155d,Winter Tuition Due Date,,https://calendar.byu.edu/other/winter-tuition-due-date-2021-12-27,e9c03db7,2021-12-27 00:00:00,,05729b31
4d332a73,M Basketball vs. Westminster,"Marriott Center, Provo, UT",https://calendar.byu.edu/athletics/m-basketball-vs-westminster-202...,e9c03db7,2021-12-29 00:00:00,,05729b31
4880b650,W Basketball vs. San Diego,"Marriott Center, Provo, UT",https://calendar.byu.edu/athletics/w-basketball-vs-san-diego-2021-...,e9c03db7,2021-12-30 00:00:00,,05729b31


## Finished events table 

In [296]:
# Combine events scraped from utah.com and byu.edu into one dataframe
dfEvents = dfEvents_2.append(dfEvents2)
dfEvents

,name,description,url,city_id,start_date,end_date,source_id
event_id,,,,,,,
d78f7a1d,Saturday Morning Nature Walk,"Learn about the beautiful wetlands, native plant and animal specie...",https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...,5f3b344b,2021-12-10 00:00:00,2021-12-18 00:00:00,bbdadd3b
56758989,Sundance Film Festival,"The 10-day festival brings more than 40,000 people to Park City ea...",https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...,5f3b344b,2022-01-20 00:00:00,,bbdadd3b
dd0f8a10,Luminaria: Experience the Light,This exciting mile walk is filled with thousands of programmed lum...,https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...,bd85c73f,2022-11-18 00:00:00,2022-01-01 00:00:00,bbdadd3b
3ee09b1b,Spanish Fork Festival of Lights,Located at Canyon View Park. Admission is $8 per single-family veh...,https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...,39cddece,2021-11-25 00:00:00,2022-01-01 00:00:00,bbdadd3b
3f7f511d,Heber Valley Polar Express Train,Join us on this fun and traditional train ride to the North Pole! ...,https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...,2dc1839d,2022-11-29 00:00:00,2021-12-23 00:00:00,bbdadd3b
65aae7a2,Holiday Window Stroll at The Grand America,Experience the holiday windows! Join Clara and the Nutcracker for ...,https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...,7bf336c7,2021-12-10 00:00:00,2022-01-01 00:00:00,bbdadd3b
5a2ce057,Christmas at the Ranch,"You'll see Santa's Workshop, Santa's Supply Train, the Candy Cane ...",https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...,7bf336c7,2022-11-26 00:00:00,2021-12-23 00:00:00,bbdadd3b
50cbb998,Beyond Van Gogh: The Immersive Experience,"While journeying through Beyond Van Gogh, guests witness over 300 ...",https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...,7bf336c7,2021-12-10 00:00:00,2021-12-31 00:00:00,bbdadd3b
a000289b,Dreamscapes - Immersive Art Exhibit,Don’t miss this limited opportunity to dream with your eyes open a...,https://utah.com/go?page_id=58e6bf383c0330e67d744a04&page_type=pag...,7bf336c7,2021-12-10 00:00:00,,bbdadd3b


## Final cleaning to city table, and creation of source table

In [297]:
# remove alternative city names from city table
alternativeCityDf = dfCities[dfCities.city_name.isin(list(alternativeNames.values()))]
dfCities = dfCities.drop(alternativeCityDf.index)
dfCities.set_index("city_id", inplace=True)
dfCities

,city_name
city_id,
e9c03db7,Provo
39cddece,Spanish Fork
e6ec6a8f,Payson
074cbda1,Springville
86e7b951,Orem
7bf336c7,Salt Lake City
bbac1723,Draper
bd85c73f,Lehi
e4848169,American Fork


In [298]:
# Create source dataframe
data = {"source_id":[sourceidUtah, sourceidBYU], "name":[nameUtah, nameBYU],"url":[urlUtah, urlBYU]}

dfSource = pd.DataFrame(data)
dfSource.set_index("source_id", inplace=True)
dfSource

,name,url
source_id,,
bbdadd3b,utah.com,https://utah.com/events-in-utah
05729b31,byu,https://calendar.byu.edu/events


## Input data into mysql database

In [299]:
def insertIntoMySQLDb(data, tableName, database = "event_tracker"):
    # data should be a list of tuples
    # order of columns in mysql must be the same as those in the dataframe
    # must input the primary keys first before adding their associated foreign keys
    
    import mysql.connector
    import time

    # get database access credentials from user
    print("Enter the user:")
    time.sleep(1)
    username = input()

    print("Enter the password:")
    time.sleep(1)
    password = input()

    print("Enter the host ip address:")
    time.sleep(1)
    hostIp = input()


    # connect to mysql database
    try:
        cnx = mysql.connector.connect(user=username, password=password,
                                    host=hostIp,
                                    database=database)
    except:
        print("Failed to connect")
        return


    # get the table column names
    mycursor = cnx.cursor()
    sql = f"desc {tableName}"
    mycursor.execute(sql)
    myresult = mycursor.fetchall()

    # make string of column names and input positions
    cols = ""
    values =""
    for i in range(0,len(myresult)):
        if i == 0:
            cols = myresult[i][0]
            values = "%s"
        else:
            cols = cols + ", " + (myresult[i][0])
            values = values + ", %s"
    
    # execute the sql statement, inserting the data into the columns
    sql = f"INSERT INTO {tableName} ({cols}) VALUES ({values})"
    
    try:
        mycursor.executemany(sql, data)
        cnx.commit()
    except:
        print("error occurred")
    cnx.close()


# convert column to datetime format (cells were already in the format, just not the column)
dfEvents["start_date"] = pd.to_datetime(dfEvents.start_date, format='%Y-%m-%d')
dfEvents["end_date"] = pd.to_datetime(dfEvents.end_date, format='%Y-%m-%d')

# convert datetimes to strings so it is readable by mysql database
dfEvents["start_date"]     = dfEvents["start_date"].dt.strftime('%Y-%m-%d')
dfEvents["end_date"]       = dfEvents["end_date"].dt.strftime('%Y-%m-%d')

# If the value is null, instead input a None
dfEvents.loc[dfEvents["end_date"].isnull(), "end_date"] = None
dfEvents.loc[dfEvents["start_date"].isnull(), "start_date"] = None

# convert dataframes to lists of tuples 
eventsInput = list(dfEvents.to_records(index=True))
sourceInput = list(dfSource.to_records(index=True))
citiesInput = list(dfCities.to_records(index=True))

# insert the dataframes into MySql db
# must input the primary keys first before adding their associated foreign keys
insertIntoMySQLDb(sourceInput, "source")
insertIntoMySQLDb(citiesInput, "city")
insertIntoMySQLDb(eventsInput, "event")

Enter the user:
Enter the password:
Enter the host ip address:
Enter the user:
Enter the password:
Enter the host ip address:
Enter the user:
Enter the password:
Enter the host ip address:
